In [43]:
import requests
import polars as pl
import re

In [44]:
def get_steam_gamesIDs():
    url = f"http://api.steampowered.com/ISteamApps/GetAppList/v2/"
    response = requests.get(url)
    data = response.json()
    return data['applist']['apps']  # This is a list of dictionaries with 'appid' and 'name'

In [45]:
games_list = get_steam_gamesIDs()

In [46]:
print(len(games_list))

198468


In [54]:
def clean_app_list(games_list):
    # List of keywords to exclude
    excluded = ['test', 'client', 'server', 'soundtrack', 'demo']
    
    # RegEx pattern for checking European characters
    european_chars_pattern = re.compile(r'^[a-zA-Z0-9 \-\'!@#$%^&*()_+={}[\]|\\:;"<>,.?/~`€£±§]+$')
    
    # Filter the list by removing dictionaries whose 'name' is empty,
    # contains 'test', 'client', 'server', 'soundtrack', or non-European characters.
    filtered_games = [
        app for app in games_list 
        if app['name'] and all(exclude not in app['name'].lower() for exclude in excluded)
        and european_chars_pattern.match(app['name'])
    ]
    return filtered_games

In [55]:
filtered_games = clean_app_list(games_list)

In [57]:
print(len(filtered_games))

152214


In [60]:
filtered_games[-10:]

[{'appid': 2971390, 'name': 'Roll Five'},
 {'appid': 1392860, 'name': 'Little Nightmares III'},
 {'appid': 1372660, 'name': 'An Amazing Wizard'},
 {'appid': 2945100, 'name': 'Idle Taoist Mage Warrior 2'},
 {'appid': 1734320, 'name': 'Brutal Orchestra'},
 {'appid': 2850810, 'name': 'Sticky Clicker!'},
 {'appid': 1640370, 'name': 'The Court Of Wanderers'},
 {'appid': 2588990, 'name': 'Survive It: Frozen'},
 {'appid': 1592750, 'name': 'Anode Heart'},
 {'appid': 2085000, 'name': 'SaGa Emerald Beyond '}]

In [61]:
df_games = pl.DataFrame(filtered_games)
sorted_df_games = df_games.sort("appid")
sorted_df_games.write_csv('SteamGames.csv')

In [63]:
sorted_df_games.head(20)

appid,name
i64,str
8,"""winui2"""
10,"""Counter-Strike"""
20,"""Team Fortress Classic"""
30,"""Day of Defeat"""
40,"""Deathmatch Classic"""
…,…
218,"""Source SDK Base 2007"""
220,"""Half-Life 2"""
240,"""Counter-Strike: Source"""


In [21]:
def find_apps_by_term(apps, term):
    # Überprüfe, ob der Begriff im 'name' jedes Dictionaries enthalten ist. Ignoriere dabei Groß- und Kleinschreibung.
    matching_apps = [app for app in apps if term.lower() in app['name'].lower()]
    return matching_apps

In [64]:
search = find_apps_by_term(filtered_apps, " ")
search[:5]

[{'appid': 2744840, 'name': 'Road Toad'},
 {'appid': 2744860, 'name': 'Coloring Pixels - Space 2 Pack'},
 {'appid': 2744900, 'name': 'Detective Kiwi'},
 {'appid': 2744930, 'name': 'Flee or Be'},
 {'appid': 2744950, 'name': "Dad's Coming Demo"}]

In [67]:
def find_game_name(games, appid):
    for game in games:
        if game['appid'] == appid:
            return game['name']
    return "Spiel mit dieser AppID nicht gefunden."

In [68]:
appid = 10
find_game_name(filtered_apps, appid)

'Counter-Strike'